In [1]:
##########################################################################
# Created on Sat Nov 20 19:20:45 2021                                    #
# Python for Financial Analysis and Risk Management                      #
# @author: Meng Lipeng (FRM, CFA)                                        #
##########################################################################

# 14.3.2.Pricing European future option-Black model

### 1.Mathimatical expression

Suppose that in a risk neutral world, the underlying futures contract price F follows the following stochastic process:\
$$dF=\sigma Fdx\tag{14-12}$$
where,\
dx:Wiener process

According Black model,\
$$c=e^{-rT}[F_0N(d_1)-KN(d_2)]\tag{14-13}$$
$$p=e^{-rT}[KN(-d2)-F_0N(-d_1)]\tag{14-14}$$
$$d_1=\frac{ln(F_0/K)+\sigma ^2T/2}{\sigma \sqrt{T}}$$
$$d_2=\frac{ln(F_0/K)-\sigma ^2T/2}{\sigma \sqrt{T}}=d_1-\sigma\sqrt{T}$$
where,\
$N(\cdot)$:CDF(Cumulative Distribution Function) of standard normal distribution

### 2.Implementation

In [12]:
def Black_model(F,K,sigma,r,T,typ):
    '''Define a function to calculate European future option price using Black model.
    F:underlying future's price
    K:strike price
    sigma:volatility of future's return(annualized)
    r:risk free rate(continuous compound)
    T:tenor of option in year
    typ:option type. 'call'indicates call option, otherwise put option '''
    from numpy import log, exp, sqrt
    from scipy.stats import norm
    
    d1=(log(F/K)+pow(sigma,2)*T/2)/(sigma*sqrt(T))
    d2=d1-sigma*sqrt(T)
    if typ=='call':
        price=exp(-r*T)*(F*norm.cdf(d1)-K*norm.cdf(d2))
    else:
        price=exp(-r*T)*(K*norm.cdf(-d2)-F*norm.cdf(-d1))
    return price

### 3.Case study

In [10]:
import pandas as pd
import numpy as np

price_AU2012=pd.read_excel('AU2012(20191118-20200911).xlsx',engine='openpyxl',sheet_name="Sheet1",header=0,index_col=0)

return_AU2012=np.log(price_AU2012/price_AU2012.shift(1))
Sigma_AU2012=np.sqrt(252)*np.std(return_AU2012)
Sigma_AU2012=float(Sigma_AU2012)
print('Annualized volatility of AU2012 is ',round(Sigma_AU2012,4))

Annualized volatility of AU2012 is  0.1757


In [13]:
import datetime as dt

t0=dt.datetime(2020,9,11)
t1=dt.datetime(2020,11,24)
tenor=(t1-t0).days/365

strike=380
shibor_Sep11=0.02697
price_Sep11=420.36

price_call=Black_model(F=price_Sep11,K=strike,sigma=Sigma_AU2012,r=shibor_Sep11,T=tenor,typ='call')
price_put=Black_model(F=price_Sep11,K=strike,sigma=Sigma_AU2012,r=shibor_Sep11,T=tenor,typ='put')

print('Price of AU2012 call 380 option is ',round(price_call,4))
print('Price of AU2012 put 380 option is ',round(price_put,4))

Price of AU2012 call 380 option is  41.645
Price of AU2012 put 380 option is  1.5051


# 14.3.3.Pricing American future option-Binomial tree

### 1.One step binomial tree

Risk free portfolio(Covered call)=long h future + short one call option\
Assuming initial margin ratio of future is k\
$$V_0=hkF_0-f\tag{14-15}$$

When future option expires,\
  Situation 1：Future price rises to $F_0u$\
$$V_T=hkF_0e^{rT}+h(F_0u-F_0)-f_u\tag{14-16}$$
  Situation 2：Future price drops to $F_0d$\
$$V_T=hkF_0e^{rT}+h(F_0d-F_0)-f_d\tag{14-17}$$

According to the principle of no arbitrage, (14-16) and (14-17) are equivalent, after rearrangement, we can get
$$h=\frac{f_u-f_d}{F_0u-F_0d}\tag{14-18}$$

Using (14-16),
$$V_0=V_Te^{-rT}=hkF_0+e^{-rT}[h(F_0u-F_0)-f_u]\tag{14-19}$$
Combine (14-15) and (14-19), we can get
$$-f=e^{-rT}[h(F_0u-F_0)-f_u]\tag{14-20}$$
Substitute (14-18) to (14-20) and after rearragement, we can get
$$f=e^{-rT}[pf_u+(1-p)f_d]\tag{14-21}$$
where,\
$$p=\frac{1-d}{u-d}\tag{14-22}$$
$$1-p=\frac{u-1}{u-d}$$

### 2.N-steps binomial tree

American call option,\
$$V_{i,j}=max\left\{F_0u^jd^{i-j}-K,\left[pV_{i+1,j+1}+(1-p)V_{i+1,j}\right]e^{-r\Delta t}\right\}\tag{14-26}$$
American put option,\
$$V_{i,j}=max\left\{K-F_0u^jd^{i-j},\left[pV_{i+1,j+1}+(1-p)V_{i+1,j}\right]e^{-r\Delta t}\right\}\tag{14-27}$$

### 3.Implementation

In [15]:
def FutOption_call_Amer(F,K,sigma,r,T,N):
    '''Define a function to calculate American future call option price using N steps binomial tree.
    F:underlying future price at t0
    K:strike price
    sigma:volatility of underlying future's return(annualized)
    r:risk free rate(continuous compound)
    T:tenor in year
    N:number of step in the BTM model'''

    import numpy as np
    
    # Step1:Calculate relevant parameters
    t=T/N
    u=np.exp(sigma*np.sqrt(t))
    d=1/u
    p=(1-d)/(u-d)
    call_matrix=np.zeros((N+1,N+1))
    
    # Step2:Calculate the underlying asset price and option value at the option maturity node
    N_list=np.arange(0,N+1)
    F_end=F*pow(u,N-N_list)*pow(d,N_list)
    call_matrix[:,-1]=np.maximum(F_end-K,0)
    
    #Step3:Calculate the underlying asset price and option value at the non maturity node of the option
    i_list=list(range(0,N))
    i_list.reverse()
    for i in i_list:
        j_list=np.arange(i+1)
        Fi=F*pow(u,i-j_list)*pow(d,j_list)
        call_strike=np.maximum(Fi-K,0)
        call_nostrike=(p*call_matrix[:i+1,i+1]+(1-p)*call_matrix[1:i+2,i+1])*np.exp(-r*t)
        call_matrix[:i+1,i]=np.maximum(call_strike,call_nostrike)
    call_begin=call_matrix[0,0]
    
    return call_begin

In [16]:
def FutOption_put_Amer(F,K,sigma,r,T,N):
    '''Define a function to calculate American future put option price using N steps binomial tree.
    F:underlying future price at t0
    K:strike price
    sigma:volatility of underlying future's return(annualized)
    r:risk free rate(continuous compound)
    T:tenor in year
    N:number of step in the BTM model'''

    import numpy as np
    
    # Step1:Calculate relevant parameters
    t=T/N
    u=np.exp(sigma*np.sqrt(t))
    d=1/u
    p=(1-d)/(u-d)
    put_matrix=np.zeros((N+1,N+1))
    
    # Step2:Calculate the underlying asset price and option value at the option maturity node
    N_list=np.arange(0,N+1)
    F_end=F*pow(u,N-N_list)*pow(d,N_list)
    put_matrix[:,-1]=np.maximum(K-F_end,0)
    
    #Step3:Calculate the underlying asset price and option value at the non maturity node of the option
    i_list=list(range(0,N))
    i_list.reverse()
    for i in i_list:
        j_list=np.arange(i+1)
        Fi=F*pow(u,i-j_list)*pow(d,j_list)
        put_strike=np.maximum(K-Fi,0)
        put_nostrike=(p*put_matrix[:i+1,i+1]+(1-p)*put_matrix[1:i+2,i+1])*np.exp(-r*t)
        put_matrix[:i+1,i]=np.maximum(put_strike,put_nostrike)
    put_begin=put_matrix[0,0]
    return put_begin

### 4.Case study

In [17]:
price_M2103=pd.read_excel('Soybean meal future 2103(20200316-20201105).xlsx',engine='openpyxl',sheet_name="Sheet1",header=0,index_col=0)

return_M2103=np.log(price_M2103/price_M2103.shift(1))
Sigma_M2103=np.sqrt(252)*np.std(return_M2103)
Sigma_M2103=float(Sigma_M2103)
print('Annualized volatility of M2103 is ',round(Sigma_M2103,4))

Annualized volatility of M2103 is  0.1285


In [18]:
T_3M=3/12
strike=3000
shibor_Nov5=0.02996
price_Nov5=3221
step=100

value_Amercall=FutOption_call_Amer(F=price_Nov5,K=strike,sigma=Sigma_M2103,r=shibor_Nov5,T=T_3M,N=step)
value_Amerput=FutOption_put_Amer(F=price_Nov5,K=strike,sigma=Sigma_M2103,r=shibor_Nov5,T=T_3M,N=step)

print('Price of M2103 American call 3000 option is ',round(value_Amercall,4))
print('Price of M2103 American put 3000 option is ',round(value_Amerput,4))

Price of M2103 American call 3000 option is  233.4664
Price of M2103 American put 3000 option is  13.5038
